In [1]:
import pandas as pd
import csv

path_base="/Users/leepand/Downloads/github/rec-a-sketch-master"
df_likes = pd.read_csv(path_base+'/data/model_likes_anon.psv',
                 sep='|', quoting=csv.QUOTE_MINIMAL,
                 quotechar='\\')
df_likes.drop_duplicates(inplace=True)
df_likes.head()


sideinfo = pd.read_csv(path_base+'/data/model_feats.psv',
                       sep='|', quoting=csv.QUOTE_MINIMAL,
                       quotechar='\\')

In [2]:
sideinfo.head()

,mid,type,value
0,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,category,Characters
1,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,category,Gaming
2,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,tag,3dsmax
3,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,tag,noel
4,5dcebcfaedbd4e7b8a27bd1ae55f1ac3,tag,loli


In [3]:
#from lightfm.data import Dataset
from bolt4ds.recsys import Dataset

data = Dataset()
data.fit(df_likes.uid.unique(), df_likes.mid.unique())
num_users, num_items = data.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))


interactions_matrix,weights_matrix = data.build_interactions([(i[1],i[0])for i in df_likes.drop(['modelname'], axis = 1).values])

item_feats = data.buid_sparse_item_features(sideinfo)

/Users/leepand/miniconda3/lib/python3.7/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


Num users: 62583, num_items 28806.
(28806, 21951)


In [108]:
user_items =interactions_matrix 
t=user_items.tocsr()[0].indices

liked = set()
liked.update(t)
liked

{0, 218, 312, 3047}

In [68]:
import time
t1=time.time()
item_feature_names = getattr(item_features, 'columns', None)
item_feat_csr = item_features\
            .loc[:, item_feature_names]\
            .reindex(item_ids, axis=0)\
            .data
item_feat_csr
print(time.time()-t1)

0.023705005645751953


In [4]:
from web.py3helpers import (
    is_iter,
    iteritems,
    itervalues,
)

def dictreverse(mapping):
    """
    Returns a new dictionary with keys and values swapped.
        >>> dictreverse({1: 2, 3: 4})
        {2: 1, 4: 3}
    """
    return dict([(value, key) for (key, value) in iteritems(mapping)])

items_mapping  = data.mapping()[2]
item_ids = dictreverse(items_mapping)
idx_to_mid = item_ids

In [14]:
import bolt4ds.sparsity as sp3
import numpy as np
from bolt4ds.recsys import LFMRecommender

test=LFMRecommender(indicators="both",loss='warp', random_state=2020)

test.fit_partial(sp3.SparseFrame(interactions_matrix))
items = df_likes.mid.index.unique()
#df_likes.mid.unique()
idx_to_mid = item_ids


In [109]:
%%timeit
scores = test.predict_online(user_id=600000000, item_ids=np.arange(n_items), item_features=None,
                             user_features=None, num_threads=5, use_baseline=False)

i_idx = [x for x in np.argsort(-scores)]
scores
i_idx[1:20]

1.25 s ± 49.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [62]:
#%%timeit
scores = test.predict(0, np.arange(n_items),
                               None, None,
                               4)
scores
i_idx = [x for x in np.argsort(-scores)]
i_idx[1:20]

[0, 13, 2, 6, 7, 10, 3, 12, 8, 16, 11, 17, 30, 5, 33, 15, 14, 37, 9]

In [58]:
#%%timeit
scores2 = test2.predict(2, list(idx_to_mid.keys()), item_feats,
                             None, 3)

scores2

array([ 5.00066233,  3.34672093,  4.5518322 , ...,  1.24682033,
       -0.84083492, -0.44999784])

In [59]:
[x for x in np.argsort(-scores2)][1:20]

[589,
 1415,
 126,
 178,
 4318,
 545,
 252,
 197,
 3189,
 16411,
 658,
 2589,
 469,
 42,
 524,
 142,
 2029,
 2828,
 310]

In [55]:
[x for x in np.argsort(-scores3)][1:20]

[589,
 178,
 658,
 126,
 0,
 2029,
 9865,
 1415,
 545,
 16411,
 361,
 42,
 142,
 37,
 269,
 197,
 2589,
 24509,
 113]

In [53]:
#scores3

0        5.959468
1        5.257864
2        5.363543
3        3.776401
4        4.671180
           ...   
28801   -3.117323
28802   -1.069238
28803    1.461367
28804   -1.038598
28805   -0.280147
Length: 28806, dtype: float64

In [6]:
test2=LFMRecommender(indicators="both",loss='warp', random_state=2020)
item_features= sp3.SparseFrame(item_feats)
test2.fit_partial(sp3.SparseFrame(interactions_matrix),item_features=item_features)


In [57]:
#%%timeit
import time
t1=time.time()
scores3 = test2.predict_online(user_id=2, item_ids= list(idx_to_mid.keys()), item_features=item_features,
                             user_features=None, num_threads=3, use_baseline=False)

scores3
print(time.time()-t1)

0.5223119258880615


In [30]:
n_items=len(list(idx_to_mid.keys()))
np.arange(n_items)

array([    0,     1,     2, ..., 28803, 28804, 28805])

In [27]:
test2.item_embeddings

array([[ 0.05760053,  0.04967599,  0.04583591, ...,  0.03500086,
        -0.14928071, -0.06391397],
       [ 0.08932079,  0.14446905,  0.05778684, ..., -0.00580893,
        -0.0772007 ,  0.07091177],
       [-0.0582766 ,  0.00451397, -0.0826065 , ..., -0.1185465 ,
         0.0602765 , -0.13355203],
       ...,
       [-0.05219976, -0.23332356,  0.12589337, ...,  0.11890989,
        -0.05048256, -0.26423895],
       [-0.07222652, -0.24586554,  0.00446317, ..., -0.2949972 ,
         0.4593784 ,  0.34493247],
       [-0.17191826, -0.09447052,  0.0347207 , ...,  0.267723  ,
        -0.10988855,  0.03161732]], dtype=float32)

In [74]:
import numpy as np
from lightfm import LightFM

class TrainLightFM:
    def __init__(self):
        pass
        
    def train_test_split(self, interactions, weights):
        train_interactions, test_interactions = \
        cross_validation.random_train_test_split(
            interactions, 
            random_state=np.random.RandomState(2019))
        
        train_weights, test_weights = \
        cross_validation.random_train_test_split(
            weights, 
            random_state=np.random.RandomState(2019))
        return train_interactions,\
    test_interactions, train_weights, test_weights
    
    def fit(self, interactions, weights,
            item_features, user_features,
            cross_validation=False,no_components=150,
            learning_rate=0.05,
            loss='warp',
            random_state=2019,
            verbose=True,
            num_threads=4, epochs=5):
        ################################
        # Model building part
        ################################

        # define lightfm model by specifying hyper-parametre
        # then fit the model with ineteractions matrix,
        # item and user features
        
        model = LightFM(
            no_components,
            learning_rate,
            loss=loss,
            random_state=random_state)
        model.fit(
            interactions,
            item_features=item_features,
            user_features=user_features, sample_weight=weights,
            epochs=epochs, num_threads=num_threads, verbose=verbose)
        
        return model

In [75]:
train_lightfm = TrainLightFM()

In [76]:
# finally build and trian our model
interactions = interactions_matrix
weights = weights_matrix
questions_features = item_feats
professional_features =None
model = train_lightfm.fit(interactions,
                          weights,
                          questions_features,
                          professional_features)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4


In [95]:
#%%timeit
professional_id = 1000000000
score = model.predict(
    professional_id,
    list(idx_to_mid.keys()), 
    item_features=questions_features,
    user_features=None)
        


ValueError: The user feature matrix specifies more features than there are estimated feature embeddings: 62583 vs 1000000001.

In [82]:
num_prediction = 10
import pandas as pd
questions_for_prediction=pd.Series([])
questions_for_prediction['recommendation_score'] = score
questions_for_prediction = questions_for_prediction.sort_values(
    ascending=False)[:num_prediction]
questions_for_prediction

/Users/leepand/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until


recommendation_score    [9.977973937988281, 5.214651584625244, 1.81977...
dtype: object

In [80]:
questions_for_prediction

recommendation_score    [9.977973937988281, 5.214651584625244, 1.81977...
dtype: object

In [83]:
score

array([ 9.97797394,  5.21465158,  1.81977296, ..., -3.61874223,
       -1.08967376, -1.40748131])

In [84]:
[x for x in np.argsort(-score)][1:20]

[671,
 8835,
 38,
 28,
 9287,
 4421,
 10501,
 573,
 46,
 740,
 208,
 959,
 1217,
 3822,
 1775,
 93,
 4571,
 3241,
 7214]

In [90]:
score[7214]

10.079724311828613

In [ ]:
class LightFMRecommendations:
    """
    Make prediction given model and professional ids
    """
    def __init__(self, lightfm_model,
                 professionals_features,
                 questions_features,
                 questions,professionals,merge):
        self.model = lightfm_model
        self.professionals_features = professionals_features
        self.questions_features = questions_features
        self.questions = questions
        self.professionals = professionals
        self.merge = merge
        
    def previous_answered_questions(self, professionals_id):
        previous_q_id_num = (
            self.merge.loc[\
                self.merge['professionals_id_num'] == \
                professionals_id]['questions_id_num'])
        
        previous_answered_questions = self.questions.loc[\
            self.questions['questions_id_num'].isin(
            previous_q_id_num)]
        return previous_answered_questions
        
    
    def _filter_question_by_pro(self, professionals_id):
        """Drop questions that professional already answer"""
        previous_answered_questions = \
        self.previous_answered_questions(professionals_id)
        
        discard_qu_id = \
        previous_answered_questions['questions_id_num'].values.tolist()
        
        questions_for_prediction = \
        self.questions.loc[~self.questions['questions_id_num'].isin(discard_qu_id)]
        
        return questions_for_prediction
    
    def _filter_question_by_date(self, questions, start_date, end_date):
        mask = \
        (questions['questions_date_added'] > start_date) & \
        (questions['questions_date_added'] <= end_date)
        
        return questions.loc[mask]
        
    
    def recommend_by_uid_general(self,
                                    professional_id,
                                    num_prediction=8):
        questions_for_prediction = self._filter_question_by_pro(professional_id)
        score = self.model.predict(
            professional_id,
            questions_for_prediction['questions_id_num'].values.tolist(), 
            item_features=self.questions_features,
            user_features=self.professionals_features)
        
        questions_for_prediction['recommendation_score'] = score
        questions_for_prediction = questions_for_prediction.sort_values(
            by='recommendation_score', ascending=False)[:num_prediction]
        return questions_for_prediction
    
    def recommend_by_uid_frequency_date_range(self,
                                                 professional_id,
                                                 start_date,
                                                 end_date,
                                                 num_prediction=8):
        questions_for_prediction = \
        self._filter_question_by_pro(professional_id)
        
        start_date = datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.strptime(end_date, '%Y-%m-%d')
        
        questions_for_prediction = self._filter_question_by_date(
            questions_for_prediction, start_date, end_date)
        
        score = self.model.predict(
            professional_id,
            questions_for_prediction['questions_id_num'].values.tolist(), 
            item_features=self.questions_features,
            user_features=self.professionals_features)
        
        questions_for_prediction['recommendation_score'] = score
        questions_for_prediction = questions_for_prediction.sort_values(
            by='recommendation_score', ascending=False)[:num_prediction]
        return questions_for_prediction